In [ ]:
# Imports
from google.colab import drive
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

drive.mount("/content/drive/")

!cd /content/drive/Shareddrives/'Replicon Data Source'/'Data Sources'

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
## Allows access to google drive

# Import Google libraries
import gspread
from google.colab import auth
from oauth2client.client import GoogleCredentials
from google.auth import default

# Authenticate and create a client
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

# Open the Google Sheets file
spreadsheet = gc.open("Salary Activity Report").sheet1

# 'worksheet' object, fetching the data
data = spreadsheet.get_all_values()

In [ ]:
# Put that data into a Pandas dataframe
df = pd.DataFrame(data)
df.columns = df.iloc[0]
df = df[1:]

In [ ]:
# Creating a boolean list variable "key" where df records are null
columns_to_check = ['VES DBQ + IMO total', 'LS DBQ + IMO total', 'QTC DBQ + IMO Total', 'Optum DBQ + IMO total', 'RHRP Eval total']
df['activity_sums'] = df[columns_to_check].sum(axis=1)
df['activity_sums'] = pd.to_numeric(df['activity_sums'], errors='coerce')
df['activity_sums'] = df['activity_sums'].fillna(0)

df['Hours'] = pd.to_numeric(df['Hours'], errors='coerce')
df['Hours'] = df['Hours'].fillna(0)

In [ ]:
#key = df ["Activity Name"].isnull()
hours_key = (df["Hours"] > 0)
activity_key = df['activity_sums'] > 0

Activity_df = df[activity_key]
Hours_df = df[hours_key]
##tilde used to invert key

In [ ]:
Hours_df = Hours_df[["Entry Date","User Name", "Activity Name", "Employee Title","Hours"]]

Hours_df = Hours_df.groupby(["Entry Date","User Name"]).agg({
    "Activity Name": lambda x: ', '.join(set(x)),
    "Employee Title": lambda x: ', '.join(set(x)),
    "Hours": "sum"
}).reset_index()

# Convert 'Entry Date' in Hours_df to datetime64[ns]
Hours_df["Entry Date"] = pd.to_datetime(Hours_df["Entry Date"])

In [ ]:
# Columns to check
columns_to_check = ['VES DBQ + IMO total', 'LS DBQ + IMO total', 'QTC DBQ + IMO Total', 'Optum DBQ + IMO total', 'RHRP Eval total']

# Convert act columns to ints
Activity_df[columns_to_check] = Activity_df[columns_to_check].apply(pd.to_numeric, errors='coerce')
Activity_df[columns_to_check] = Activity_df[columns_to_check].fillna(0)

# Convert to int
for col in columns_to_check:
    Activity_df[col] = Activity_df[col].astype(int)

In [ ]:
# Function to find the first non-null column name
def find_non_zero_column(row):
    cols = []
    for col in columns_to_check:
        if (row[col]) > 0:
            cols.append(col)
    return ', '.join(cols)

# Function to find filled activity counts
def find_non_zero_activity_number(row):
    values = []
    for col in columns_to_check:
        if (row[col]) > 0:
            values.append(str(row[col]))
    return ', '.join(values)

# Function to find filled activity sums
def find_non_zero_activity_sums(row):
    value = 0
    for col in columns_to_check:
        if (row[col]) > 0:
            value += row[col]
    return value

# Apply the function to each row and create a new column
Activity_df['VA_Company'] = Activity_df.apply(find_non_zero_column, axis=1)

Activity_df['activity_count'] = Activity_df.apply(find_non_zero_activity_number, axis=1)

Activity_df['activity_sums'] = Activity_df.apply(find_non_zero_activity_sums, axis=1)

Activity_df_output = Activity_df.copy()

In [ ]:
# Remove unneeded columns
Activity_df =Activity_df[["Entry Date","User Name","Employee Title","VA_Company","activity_count", "activity_sums"]]

# Convert 'Entry Date' in Activity_df to datetime64[ns]
Activity_df["Entry Date"] = pd.to_datetime(df["Entry Date"], dayfirst=False)

In [ ]:
output_df = pd.merge(Hours_df, Activity_df, on=["Entry Date","User Name","Employee Title"], how="outer")

output_df['Hours'] = output_df['Hours'].fillna(0)
output_df['activity_sums'] = output_df['activity_sums'].fillna(0)

In [ ]:
# Create csv file with copmpleted output
#output_df.to_csv('/content/drive/Shareddrives/Replicon Data Source/Data Sources/Cleaned_Salary_Report_Output.csv', index=False)

In [ ]:
# Fill NA values with blank strings because sheets doesn't like that
output_df = output_df.fillna('')

# Convert 'Entry Date' to string format
output_df['Entry Date'] = output_df['Entry Date'].dt.strftime("%-d-%b-%y")


#Open specified google sheets WB.Open sheet1.Clear the contents
worksheet = gc.open('Cleaned_Salary_Report_Output').sheet1
worksheet.clear()

# Fill sheet with dataframe contents
worksheet.update([output_df.columns.values.tolist()] + output_df.values.tolist(), raw=False)


{'spreadsheetId': '1cjNYSBarKxqE9ziKe0DsXpFkCv9EA56CIf_K5Ydh_pA',
 'updatedRange': 'Cleaned_Salary_Report_Output!A1:H687',
 'updatedRows': 687,
 'updatedColumns': 8,
 'updatedCells': 5496}